# 17th_2007

In [74]:
# =============================================================================
# 17대 대통령선거 데이터 전처리 함수들
# =============================================================================

import pandas as pd
import re
import requests
import tempfile
import os

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : 깃허브 파일 URL

    Returns:
    --------
    raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_17th_single_file(file_path, region_name):
    """
    17대 대선 개별 파일 전처리 함수

    엑셀 파일 구조:
    - 0행: 헤더 첫 번째 줄
    - 1행: 헤더 두 번째 줄
    - 2행: 헤더 세 번째 줄
    - 3행부터: 실제 데이터

    Parameters:
    -----------
    file_path : 파일 경로 (로컬 파일 또는 URL)
    region_name : 해당 파일의 지역명 (시도명)

    Returns:
    --------
    전처리된 선거 데이터 (pandas DataFrame)
    """

    # ================================
    # 1. 파일 로드
    # ================================
    if file_path.startswith(('http://', 'https://')):
        # URL인 경우
        raw_url = convert_github_url_to_raw(file_path)
        df_all = pd.read_excel(raw_url, header=None)
    else:
        # 로컬 파일인 경우
        df_all = pd.read_excel(file_path, header=None)

    # ================================
    # 2. 헤더 구성 (0~2행 병합)
    # ================================
    header_row_1 = df_all.iloc[0].fillna("")  # 첫 번째 헤더 행
    header_row_2 = df_all.iloc[1].fillna("")  # 두 번째 헤더 행
    header_row_3 = df_all.iloc[2].fillna("")  # 세 번째 헤더 행

    # 3개 헤더 행을 "_"로 연결하여 통합 헤더 생성
    combined_headers = (
        header_row_1.astype(str) + "_" +
        header_row_2.astype(str) + "_" +
        header_row_3.astype(str)
    )

    # ================================
    # 3. 실제 데이터 추출 (3행부터)
    # ================================
    df_data = df_all.iloc[3:].copy()
    df_data.columns = combined_headers

    # ================================
    # 4. 합계 행 필터링
    # ================================
    dong_column = [col for col in df_data.columns if "읍면동명" in col][0]
    filter_mask = df_data[dong_column].astype(str).str.contains("합계", na=False)
    df_filtered = df_data[filter_mask].copy()

    # ================================
    # 5. 수치형 컬럼 정리
    # ================================
    numeric_columns = df_filtered.columns[3:]

    for col in numeric_columns:
        df_filtered[col] = (
            df_filtered[col]
            .astype(str)
            .fillna("0")
            .str.replace(",", "")
            .str.strip()
            .replace("nan", "0")
            .replace("", "0")
            .astype(int)
        )

    # ================================
    # 6. 시도명 추가
    # ================================
    df_filtered.insert(0, "시도", region_name)

    return df_filtered


def process_17th_presidential_election_batch(user, repo, folder="original/Presidential_Elections/17th_2007"):
    """
    17대 대선 전체 파일 자동 처리 및 병합 함수

    GitHub API를 통해 17대 대선 폴더의 모든 .xls 파일을 자동으로
    다운로드하고 전처리한 후 하나의 DataFrame으로 병합

    Parameters:
    -----------
    user : GitHub 사용자명
    repo : GitHub 저장소명
    folder : 17대 대선 파일들이 있는 폴더 경로

    Returns:
    --------
    병합된 전체 선거 데이터 (pandas DataFrame)

    Example:
    --------
    df = process_17th_presidential_election_batch("username", "korean-elections")
    """

    print(f"17대 대선 데이터 일괄 처리 시작")
    print(f"저장소: {user}/{repo}")
    print(f"폴더: {folder}")

    # ================================
    # 1. GitHub API로 파일 목록 가져오기
    # ================================
    api_url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()
        files = response.json()
        print(f"폴더에서 {len(files)}개 파일 발견")
    except Exception as e:
        print(f"API 요청 실패: {e}")
        return None

    # ================================
    # 2. 각 .xls 파일 처리
    # ================================
    processed_dfs = []
    success_count = 0

    for file_info in files:
        file_name = file_info["name"]

        # .xls 파일만 처리
        if not file_name.endswith(".xls"):
            continue

        # 파일명에서 지역명 추출 (예: "01_17대선_투표구별개표자료_서울.xls" → "서울")
        match = re.match(r"^(\d+)_.*_([^_]+)\.xls$", file_name)
        if not match:
            print(f"파일명 패턴 불일치, 무시: {file_name}")
            continue

        number, region = match.group(1), match.group(2)

        print(f"처리 중: {file_name} (지역: {region})")

        # raw URL 생성
        raw_url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{folder}/{file_name}"

        try:
            # 임시 파일로 다운로드 후 처리
            with tempfile.NamedTemporaryFile(delete=False, suffix=".xls") as tmp:
                tmp.write(requests.get(raw_url).content)
                tmp_path = tmp.name

            # 개별 파일 전처리
            df_region = process_17th_single_file(tmp_path, region)
            processed_dfs.append(df_region)

            # 임시 파일 삭제
            os.remove(tmp_path)

            success_count += 1
            print(f"  완료: {df_region.shape[0]}행 추가")

        except Exception as e:
            print(f"  오류 발생: {e}")
            continue

    # ================================
    # 3. 모든 DataFrame 병합
    # ================================
    if processed_dfs:
        df_combined = pd.concat(processed_dfs, ignore_index=True)
        print(f"")
        print(f"일괄 처리 완료:")
        print(f"  - 성공: {success_count}개 파일")
        print(f"  - 실패: {len([f for f in files if f['name'].endswith('.xls')]) - success_count}개 파일")
        print(f"  - 최종 크기: {df_combined.shape[0]}행 x {df_combined.shape[1]}열")
        return df_combined
    else:
        print("처리된 파일이 없습니다.")
        return None


def add_17th_totals_by_region(df):
    """
    17대 대선 데이터에 시도별 및 전국 합계 추가 함수

    시도별 합계 행을 각 시도 데이터 위에 추가하고,
    전국 합계 행을 맨 위에 추가

    Parameters:
    -----------
    df : 17대 대선 원본 데이터 (DataFrame)

    Returns:
    --------
    합계 행이 추가된 DataFrame

    Example:
    --------
    df_with_totals = add_17th_totals_by_region(df_17th)
    """

    print("시도별 및 전국 합계 행 추가 시작")

    df = df.copy()

    # ================================
    # 1. 시도별 합계 행 생성
    # ================================
    sido_order = df['시도'].drop_duplicates().tolist()
    print(f"처리할 시도 수: {len(sido_order)}")

    df_list = []

    for sido in sido_order:
        # 해당 시도 데이터
        sido_data = df[df['시도'] == sido]

        # 수치형 컬럼들의 합계 계산
        numeric_cols = sido_data.select_dtypes(include='number').columns
        agg_row = sido_data[numeric_cols].sum(numeric_only=True)
        agg_row['시도'] = sido
        agg_row['구시군명__'] = '합계'

        # 누락된 열을 빈 문자열로 채움
        for col in df.columns:
            if col not in agg_row:
                agg_row[col] = ''

        # 열 순서 맞춤
        agg_row = agg_row[df.columns.tolist()]

        # 합계 행을 해당 시도 데이터 위에 추가
        df_list.append(pd.concat([pd.DataFrame([agg_row]), sido_data], ignore_index=True))

    df_with_sido_totals = pd.concat(df_list, ignore_index=True)

    # ================================
    # 2. 전국 합계 행 생성
    # ================================
    # 시도별 합계 행들만 추출하여 전국 합계 계산
    sido_total_rows = df_with_sido_totals[df_with_sido_totals['구시군명__'] == '합계']
    national_sum = sido_total_rows.select_dtypes(include='number').sum(numeric_only=True)
    national_sum['시도'] = '전국'
    national_sum['구시군명__'] = '합계'

    # 누락된 열 처리
    for col in df.columns:
        if col not in national_sum:
            national_sum[col] = ''
    national_sum = national_sum[df.columns.tolist()]

    # ================================
    # 3. 전국 합계를 맨 위에 삽입
    # ================================
    final_df = pd.concat([pd.DataFrame([national_sum]), df_with_sido_totals], ignore_index=True)

    print(f"합계 행 추가 완료:")
    print(f"  - 시도별 합계: {len(sido_order)}행")
    print(f"  - 전국 합계: 1행")
    print(f"  - 최종 크기: {final_df.shape[0]}행 x {final_df.shape[1]}열")

    return final_df


# =============================================================================
# 사용 예시
# =============================================================================

# 17대 대선 전체 파일 자동 처리
# df_17th_raw = process_17th_presidential_election_batch("your_username", "korean-elections")

# 시도별 및 전국 합계 추가
# df_17th_final = add_17th_totals_by_region(df_17th_raw)

# 결과 확인
# print(df_17th_final.head(10))
# print("전국 합계:", df_17th_final[df_17th_final['시도'] == '전국'].shape[0], "행")
# print("시도별 데이터:", df_17th_final['시도'].value_counts())

In [75]:
# 17대 대선 전체 파일 자동 처리
df_17th_raw = process_17th_presidential_election_batch("sw1kwon", "korean-elections")

# 시도별 및 전국 합계 추가
df_17th = add_17th_totals_by_region(df_17th_raw)


17대 대선 데이터 일괄 처리 시작
저장소: sw1kwon/korean-elections
폴더: original/Presidential_Elections/17th_2007
폴더에서 16개 파일 발견
처리 중: 01_17대선_투표구별개표자료_서울.xls (지역: 서울)
  완료: 25행 추가
처리 중: 02_17대선_투표구별개표자료_부산.xls (지역: 부산)
  완료: 16행 추가
처리 중: 03_17대선_투표구별개표자료_대구.xls (지역: 대구)
  완료: 8행 추가
처리 중: 04_17대선_투표구별개표자료_인천.xls (지역: 인천)
  완료: 10행 추가
처리 중: 05_17대선_투표구별개표자료_광주.xls (지역: 광주)
  완료: 5행 추가
처리 중: 06_17대선_투표구별개표자료_대전.xls (지역: 대전)
  완료: 5행 추가
처리 중: 07_17대선_투표구별개표자료_울산.xls (지역: 울산)
  완료: 5행 추가
처리 중: 08_17대선_투표구별개표자료_경기.xls (지역: 경기)
  완료: 44행 추가
처리 중: 09_17대선_투표구별개표자료_강원.xls (지역: 강원)
  완료: 18행 추가
처리 중: 10_17대선_투표구별개표자료_충북.xls (지역: 충북)
  완료: 13행 추가
처리 중: 11_17대선_투표구별개표자료_충남.xls (지역: 충남)
  완료: 16행 추가
처리 중: 12_17대선_투표구별개표자료_전북.xls (지역: 전북)
  완료: 15행 추가
처리 중: 13_17대선_투표구별개표자료_전남.xls (지역: 전남)
  완료: 22행 추가
처리 중: 14_17대선_투표구별개표자료_경북.xls (지역: 경북)
  완료: 24행 추가
처리 중: 15_17대선_투표구별개표자료_경남.xls (지역: 경남)
  완료: 20행 추가
처리 중: 16_17대선_투표구별개표자료_제주.xls (지역: 제주)
  완료: 2행 추가

일괄 처리 완료:
  - 성공: 16개 파일
  - 실패: 0개 파일
  - 최종 크기: 248행 x 20열


In [76]:
df_17th

,시도,구시군명__,읍면동명__,투표구명__,선거인수__,투표용지\n교부수__,투표수__,후보자별 득표상황_대통합민주신당_정동영,_한나라당_이명박,_민주노동당_권영길,_민주당_이인제,_창조한국당_문국현,_참주인연합_정근모,_경제공화당_허경영,_새시대참사람연합_전관,_한국사회당_금민,_무소속_이회창,_계_,무효\n투표수__,기권수__
0,전국,합계,,,37653518,23733396,23732854,6174681,11492389,712121,160708,1375498,15380,96756,7161,18223,3559963,23612880,119974,13920664
1,서울,합계,,,8051696,5066136,5066022,1237812,2689162,116344,23214,358781,3013,22405,911,3501,596226,5051369,14653,2985674
2,서울,종로구,합계,NaN,133946,85483,85480,21515,45172,1889,398,5722,56,331,13,77,9984,85157,323,48466
3,서울,중구,합계,NaN,106837,66496,66496,17299,35335,1387,328,4324,33,240,9,52,7278,66285,211,40341
4,서울,용산구,합계,NaN,190910,117054,117051,25967,66096,2358,532,7544,78,486,12,77,13554,116704,347,73859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,경남,거창군,합계,NaN,50823,34593,34594,3637,19863,1708,127,1345,26,128,20,45,6982,33881,713,16229
261,경남,합천군,합계,NaN,45959,31742,31741,2815,18867,1626,122,927,26,109,17,62,6339,30910,831,14218
262,제주,합계,,,414022,252132,252111,81570,96495,10954,1674,19421,152,1273,144,344,37495,249522,2589,161911
263,제주,제주시,합계,NaN,296325,181325,181304,56798,70297,7059,1124,15142,114,929,90,227,27824,179604,1700,115021


In [77]:
df_17th.columns.tolist()

['시도',
 '구시군명__',
 '읍면동명__',
 '투표구명__',
 '선거인수__',
 '투표용지\n교부수__',
 '투표수__',
 '후보자별 득표상황_대통합민주신당_정동영',
 '_한나라당_이명박',
 '_민주노동당_권영길',
 '_민주당_이인제',
 '_창조한국당_문국현',
 '_참주인연합_정근모',
 '_경제공화당_허경영',
 '_새시대참사람연합_전관',
 '_한국사회당_금민',
 '_무소속_이회창',
 '_계_',
 '무효\n투표수__',
 '기권수__']

In [78]:
rename_dict17 = {
    '구시군명__': '구시군',
    '선거인수__': '선거인수',
    '투표수__': '투표수',
    '후보자별 득표상황_대통합민주신당_정동영': '득표수_1_대통합민주신당_정동영',
    '_한나라당_이명박': '득표수_2_한나라당_이명박',
    '_민주노동당_권영길': '득표수_3_민주노동당_권영길',
    '_민주당_이인제': '득표수_4_민주당_이인제',
    '_창조한국당_문국현': '득표수_6_창조한국당_문국현',
    '_참주인연합_정근모': '득표수_7_참주인연합_정근모',
    '_경제공화당_허경영': '득표수_8_경제공화당_허경영',
    '_새시대참사람연합_전관': '득표수_9_새시대참사람연합_전관',
    '_한국사회당_금민': '득표수_10_한국사회당_금민',
    '_무소속_이회창': '득표수_11_무소속_이회창',
    '_계_': '득표수_계',
    '무효\n투표수__': '무효투표수',
    '기권수__': '기권수'
}

In [79]:
df_17th.rename(columns=rename_dict17).drop(columns=['읍면동명__', '투표구명__', '투표용지\n교부수__'])

,시도,구시군,선거인수,투표수,득표수_1_대통합민주신당_정동영,득표수_2_한나라당_이명박,득표수_3_민주노동당_권영길,득표수_4_민주당_이인제,득표수_6_창조한국당_문국현,득표수_7_참주인연합_정근모,득표수_8_경제공화당_허경영,득표수_9_새시대참사람연합_전관,득표수_10_한국사회당_금민,득표수_11_무소속_이회창,득표수_계,무효투표수,기권수
0,전국,합계,37653518,23732854,6174681,11492389,712121,160708,1375498,15380,96756,7161,18223,3559963,23612880,119974,13920664
1,서울,합계,8051696,5066022,1237812,2689162,116344,23214,358781,3013,22405,911,3501,596226,5051369,14653,2985674
2,서울,종로구,133946,85480,21515,45172,1889,398,5722,56,331,13,77,9984,85157,323,48466
3,서울,중구,106837,66496,17299,35335,1387,328,4324,33,240,9,52,7278,66285,211,40341
4,서울,용산구,190910,117051,25967,66096,2358,532,7544,78,486,12,77,13554,116704,347,73859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,경남,거창군,50823,34594,3637,19863,1708,127,1345,26,128,20,45,6982,33881,713,16229
261,경남,합천군,45959,31741,2815,18867,1626,122,927,26,109,17,62,6339,30910,831,14218
262,제주,합계,414022,252111,81570,96495,10954,1674,19421,152,1273,144,344,37495,249522,2589,161911
263,제주,제주시,296325,181304,56798,70297,7059,1124,15142,114,929,90,227,27824,179604,1700,115021


In [80]:
df_17th = df_17th.rename(columns=rename_dict17).drop(columns=['읍면동명__', '투표구명__', '투표용지\n교부수__'])

In [81]:
df_17th[df_17th['구시군'] == '합계']

,시도,구시군,선거인수,투표수,득표수_1_대통합민주신당_정동영,득표수_2_한나라당_이명박,득표수_3_민주노동당_권영길,득표수_4_민주당_이인제,득표수_6_창조한국당_문국현,득표수_7_참주인연합_정근모,득표수_8_경제공화당_허경영,득표수_9_새시대참사람연합_전관,득표수_10_한국사회당_금민,득표수_11_무소속_이회창,득표수_계,무효투표수,기권수
0,전국,합계,37653518,23732854,6174681,11492389,712121,160708,1375498,15380,96756,7161,18223,3559963,23612880,119974,13920664
1,서울,합계,8051696,5066022,1237812,2689162,116344,23214,358781,3013,22405,911,3501,596226,5051369,14653,2985674
27,부산,합계,2843063,1765231,236708,1018715,48901,4599,94285,770,7351,406,1198,346319,1759252,5979,1077832
44,대구,합계,1896866,1267969,75932,876719,25777,1847,50514,367,3375,221,727,228199,1263678,4291,628897
53,인천,합계,2005874,1210220,286565,593283,42069,7612,84814,797,5769,323,1068,183057,1205357,4863,795654
64,광주,합계,1031333,663338,527588,56875,13597,7118,31524,398,1547,96,289,22520,661552,1786,367995
70,대전,합계,1098977,680264,159700,246008,17207,7223,48143,467,2356,204,683,195957,677948,2316,418713
76,울산,합계,806423,521216,70736,279891,43607,1884,28605,271,2099,121,467,90905,518586,2630,285207
82,경기,합계,8222124,5035641,1181936,2603443,144830,30513,354492,3378,23554,1312,3207,670742,5017407,18234,3186483
127,강원,합계,1164655,728895,136668,376004,28129,6085,42552,615,5063,436,849,127102,723503,5392,435760


In [82]:
df_17th.to_csv("temp1_president_17.csv", index=False, encoding="utf-8-sig")